# Notebook for EDA on WikiData

### Config for blob storage

In [0]:
storage_account = 'wikidatasubset'

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    dbutils.secrets.get(scope="databricks-secret-keys", key="blob")
)

### Define schema for table Item

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

item_schema = StructType([
    StructField("pageid", LongType(), True),
    StructField("ns", LongType(), True),
    StructField("title", StringType(), True),
    StructField("lastrevid", LongType(), True),
    StructField("modified", TimestampType(), True),
    StructField("id", StringType(), True)
])

In [0]:
container = 'init'
path = 'testdataset/latest-first-1gb.json'

df_item = spark.read.option("mode", "DROPMALFORMED").schema(item_schema).json(f'abfss://{container}@{storage_account}.dfs.core.windows.net/{path}')

In [0]:
df_item.head(10)

### Define schema for table Labels

In [0]:
from pyspark.sql.types import MapType, ArrayType, StructType, StructField, StringType

labels_struct = StructType([
    StructField("language", StringType(), True),
    StructField("value", StringType(), True)
])

labels_map = MapType(StringType(), labels_struct)

labels_schema = StructType([
    StructField("id", StringType(), True),
    StructField("type", StringType(), True),
    StructField("labels", labels_map, True)
])

In [0]:
container = 'init'
path = 'testdataset/latest-first-1gb.json'

df_label = spark.read.json(f'abfss://{container}@{storage_account}.dfs.core.windows.net/{path}', schema=labels_schema)

In [0]:
from pyspark.sql.functions import explode, col

df_label_final = (
    df_label.where(col("type") == 'item').select("id", explode("labels").alias("language", "struct"))
    .select("id", "language", col("struct.value").alias("label"))
)

In [0]:
df_label_final.write.format('parquet').save(f'abfss://{container}@{storage_account}.dfs.core.windows.net/preprocessed/aliases')

### Define schema for table Descriptions

In [0]:
from pyspark.sql.types import MapType, ArrayType, StructType, StructField, StringType

descriptions_struct = StructType([
    StructField("language", StringType(), True),
    StructField("value", StringType(), True)
])

descriptions_map = MapType(StringType(), descriptions_struct)

descriptions_schema = StructType([
    StructField("id", StringType(), True),
    StructField("descriptions", descriptions_map, True)
])

In [0]:
container = 'init'
path = 'testdataset/latest-first-1gb.json'

df_descriptions = spark.read.json(f'abfss://{container}@{storage_account}.dfs.core.windows.net/{path}', schema=descriptions_schema)

In [0]:
df_descriptions_final = (
    df_descriptions.select("id", explode("descriptions").alias("language", "struct"))
    .select("id", "language", col("struct.value").alias("description"))
)

In [0]:
df_descriptions_final.head(100)

### Define schema for table Aliases

In [0]:
from pyspark.sql.types import MapType, ArrayType, StructType, StructField, StringType

alias_struct = StructType([
    StructField("language", StringType(), True),
    StructField("value", StringType(), True)
])

aliases_map = MapType(StringType(), ArrayType(alias_struct))

aliases_schema = StructType([
    StructField("id", StringType(), True),
    StructField("aliases", aliases_map, True)
])

In [0]:
container = 'init'
path = 'testdataset/latest-first-1gb.json'

df_aliases = spark.read.json(f'abfss://{container}@{storage_account}.dfs.core.windows.net/{path}', schema=aliases_schema)

In [0]:
from pyspark.sql.functions import explode, col

df_aliases_final = (
    df_aliases.select("id", explode("aliases").alias("language", "array"))
    .select("id", "language", explode("array.value").alias("alias"))
)

In [0]:
df_aliases_final.head(100)

### Define schema for table Sitelinks

In [0]:
from pyspark.sql.types import MapType, ArrayType, StructType, StructField, StringType

sitelinks_struct = StructType([
    StructField("site", StringType(), True),
    StructField("title", StringType(), True),
    StructField("badges", ArrayType(StringType()), True)
])

sitelinks_map = MapType(StringType(), sitelinks_struct)

sitelinks_schema = StructType([
    StructField("id", StringType(), True),
    StructField("sitelinks", sitelinks_map, True)
])

In [0]:
container = 'init'
path = 'testdataset/latest-first-1gb.json'

df_sitelinks = spark.read.json(f'abfss://{container}@{storage_account}.dfs.core.windows.net/{path}', schema=sitelinks_schema)

In [0]:
from pyspark.sql.functions import explode, col, explode_outer

df_sitelinks_final = (
    df_sitelinks.select("id", explode("sitelinks").alias("wiki", "struct"))
    .select("id", "wiki", col("struct.title").alias("title"), explode_outer("struct.badges").alias("badges"))
)

In [0]:
df_sitelinks_final.head(100)

### Define schema for table Claims

In [0]:
from pyspark.sql.types import MapType, ArrayType, StructType, StructField, StringType

value_struct = StructType([
    StructField("id", StringType(), True)
])

datavalue_struct = StructType([
    StructField("value", value_struct, True)
])

mainsnak_struct = StructType([
    StructField("datavalue", datavalue_struct, True)
])

claims_struct = StructType([
    StructField("mainsnak", mainsnak_struct, True),
])

claims_map = MapType(StringType(), ArrayType(claims_struct))

claims_schema = StructType([
    StructField("id", StringType(), True),
    StructField("claims", claims_map, True)
])

In [0]:
container = 'init'
path = 'testdataset/latest-first-1gb.json'

df_claims = spark.read.json(f'abfss://{container}@{storage_account}.dfs.core.windows.net/{path}', schema=claims_schema)

In [0]:
from pyspark.sql.functions import explode, col, explode_outer

df_claims_final = (
    df_claims.select("id", explode("claims").alias("claim", "claim_datas"))
    .select("id", "claim", explode("claim_datas").alias("claim_data"))
    .select(col("id").alias("src"), col("claim_data.mainsnak.datavalue.value.id").alias("dst"), col("claim").alias("property"))
)

In [0]:
df_claims_final.head(100)